# ENG-HIN Translator
A Translator, to understand the implementation and working of how an Encoder and Decoder works  

### Setup



In [0]:
import pandas as pd
import numpy as np
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model

Mounting Drive


In [64]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Getting Dataset 
The dataset is a (.txt) files which contains the English Sentences followed by its Hindi translation. 

The dataset is downloaded from the website : 



In [65]:
path='/content/drive/My Drive/Eng-Hin-Translator/dataset/hin.txt'
lines= pd.read_table(path, names=['eng', 'hin','extra'])
lines.tail()

,eng,hin,extra
2773,"If you go to that supermarket, you can buy mos...",उस सूपरमार्केट में तुम लगभग कोई भी रोजाने में ...,CC-BY 2.0 (France) Attribution: tatoeba.org #6...
2774,The passengers who were injured in the acciden...,जिन यात्रियों को दुर्घटना मे चोट आई थी उन्हे अ...,CC-BY 2.0 (France) Attribution: tatoeba.org #4...
2775,"Democracy is the worst form of government, exc...","लोकतंत्र सरकार का सबसे घिनौना रूप है, अगर बाकी...",CC-BY 2.0 (France) Attribution: tatoeba.org #1...
2776,If my boy had not been killed in the traffic a...,अगर मेरा बेटा ट्रेफ़िक हादसे में नहीं मारा गया...,CC-BY 2.0 (France) Attribution: tatoeba.org #3...
2777,"When I was a kid, touching bugs didn't bother ...","जब मैं बच्चा था, मुझे कीड़ों को छूने से कोई पर...",CC-BY 2.0 (France) Attribution: tatoeba.org #2...


In [66]:
lines.drop(['extra'],axis=1,inplace=True)
lines.tail()

,eng,hin
2773,"If you go to that supermarket, you can buy mos...",उस सूपरमार्केट में तुम लगभग कोई भी रोजाने में ...
2774,The passengers who were injured in the acciden...,जिन यात्रियों को दुर्घटना मे चोट आई थी उन्हे अ...
2775,"Democracy is the worst form of government, exc...","लोकतंत्र सरकार का सबसे घिनौना रूप है, अगर बाकी..."
2776,If my boy had not been killed in the traffic a...,अगर मेरा बेटा ट्रेफ़िक हादसे में नहीं मारा गया...
2777,"When I was a kid, touching bugs didn't bother ...","जब मैं बच्चा था, मुझे कीड़ों को छूने से कोई पर..."


In [67]:
lines.shape

(2778, 2)

### Preprocessing

In [68]:
# Lowercase all characters
lines.eng=lines.eng.apply(lambda x: x.lower())
lines.hin=lines.hin.apply(lambda x: x.lower())
lines.tail()

,eng,hin
2773,"if you go to that supermarket, you can buy mos...",उस सूपरमार्केट में तुम लगभग कोई भी रोजाने में ...
2774,the passengers who were injured in the acciden...,जिन यात्रियों को दुर्घटना मे चोट आई थी उन्हे अ...
2775,"democracy is the worst form of government, exc...","लोकतंत्र सरकार का सबसे घिनौना रूप है, अगर बाकी..."
2776,if my boy had not been killed in the traffic a...,अगर मेरा बेटा ट्रेफ़िक हादसे में नहीं मारा गया...
2777,"when i was a kid, touching bugs didn't bother ...","जब मैं बच्चा था, मुझे कीड़ों को छूने से कोई पर..."


In [0]:
# Remove quotes
lines.eng=lines.eng.apply(lambda x: re.sub("'", '', x))
lines.hin=lines.hin.apply(lambda x: re.sub("'", '', x))


In [70]:
exclude = set(string.punctuation) # Set of all special characters
# Remove all the special characters
lines.eng=lines.eng.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
lines.hin=lines.hin.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
lines.tail()

,eng,hin
2773,if you go to that supermarket you can buy most...,उस सूपरमार्केट में तुम लगभग कोई भी रोजाने में ...
2774,the passengers who were injured in the acciden...,जिन यात्रियों को दुर्घटना मे चोट आई थी उन्हे अ...
2775,democracy is the worst form of government exce...,लोकतंत्र सरकार का सबसे घिनौना रूप है अगर बाकी ...
2776,if my boy had not been killed in the traffic a...,अगर मेरा बेटा ट्रेफ़िक हादसे में नहीं मारा गया...
2777,when i was a kid touching bugs didnt bother me...,जब मैं बच्चा था मुझे कीड़ों को छूने से कोई परे...


In [0]:
# Remove all numbers from text
remove_digits = str.maketrans('', '', digits)
lines.eng=lines.eng.apply(lambda x: x.translate(remove_digits))
lines.hin = lines.hin.apply(lambda x: re.sub("[०१२३४५६७८९abcdefghijklmnopqrstuvwxyz]", "", x))

In [0]:
# Remove extra spaces
lines.eng=lines.eng.apply(lambda x: x.strip())
lines.hin=lines.hin.apply(lambda x: x.strip())
lines.eng=lines.eng.apply(lambda x: re.sub(" +", " ", x))
lines.hin=lines.hin.apply(lambda x: re.sub(" +", " ", x))

In [0]:
# Add start and end tokens to target sequences
lines.hin = lines.hin.apply(lambda x : 'START_ '+ x + ' _END')

In [74]:
lines.sample(10)

,eng,hin
2564,this machine is superior in quality to that one,START_ यह मशीन उस मशीन से बेहतर है। _END
1738,these trees were planted by them,START_ ये पेड़ उन्होंने लगाए थे। _END
2327,im willing to help you if you want me to,START_ अगर तुम मदद चाह्ते हो तो मैं मदद करने क...
2105,its been ten years since we last met,START_ हमारी पिछली मुलाक़ात से अब दस साल हो गए...
1613,it isnt raining much this year,START_ इस साल ज़्यादा बारिश नहीं हो रही है। _END
1633,the house is said to be haunted,START_ लोग कहते हैं कि इस घर में भूत हैं। _END
2190,the work must be completed by tomorrow,START_ काम कल तक खतम होना होगा। _END
2198,youve got a nerve to say such a thing,START_ तुम्हारी जुर्रत कैसे हुई यह बोलने की _END
1317,have you ever been to africa,START_ आप कभी अफ़्रीका गए हुए हैं क्या _END
136,some fish fly,START_ कुछ मछलियाँ उड़ सकतीं हैं। _END


In [75]:
# Vocabulary of English
all_eng_words=set()
for eng in lines.eng:
    for word in eng.split():
        if word not in all_eng_words:
            all_eng_words.add(word)

# Vocabulary of Hindi 
all_hin_words=set()
for hin in lines.hin:
    for word in hin.split():
        if word not in all_hin_words:
            all_hin_words.add(word)
    
print(all_eng_words,'\n____\n',all_hin_words)

{'appearing', 'angles', 'feel', 'raise', 'desk', 'like', 'down', 'coins', 'neighbors', 'bakery', 'earlier', 'tonight', 'august', 'elected', 'birthday', 'work', 'went', 'opponent', 'bite', 'mail', 'youve', 'objected', 'sentence', 'wearing', 'pours', 'murdered', 'your', 'begins', 'committee', 'cry', 'ill', 'proved', 'protection', 'they', 'shaving', 'watching', 'sooner', 'lets', 'garden', 'takes', 'terminal', 'market', 'my', 'industry', 'what', 'away', 'sure', 'thinking', 'sky', 'disturbing', 'dress', 'qutub', 'read', 'free', 'taught', 'bent', 'atmosphere', 'suit', 'afraid', 'hes', 'these', 'sign', 'has', 'phrases', 'newspaper', 'cradle', 'resting', 'seeing', 'trees', 'hold', 'bugs', 'our', 'driven', 'dozens', 'ones', 'strong', 'family', 'afternoon', 'stripped', 'count', 'if', 'stuff', 'explained', 'correct', 'wood', 'police', 'ignore', 'mist', 'travels', 'young', 'fault', 'sing', 'mountain', 'daughters', 'kite', 'trouble', 'chat', 'lose', 'george', 'new', 'degree', 'remains', 'arrive', '

In [76]:
# Max Length of source sequence
lenght_list=[]
for l in lines.eng:
    lenght_list.append(len(l.split(' ')))
max_length_src = np.max(lenght_list)
max_length_src

22

In [77]:
# Max Length of target sequence
lenght_list=[]
for l in lines.hin:
    lenght_list.append(len(l.split(' ')))
max_length_tar = np.max(lenght_list)
max_length_tar

27

In [78]:
input_words = sorted(list(all_eng_words))
target_words = sorted(list(all_hin_words))
num_encoder_tokens = len(all_eng_words)
num_decoder_tokens = len(all_hin_words)
num_encoder_tokens, num_decoder_tokens

(2344, 2975)

In [79]:
num_decoder_tokens += 1 # For zero padding
num_decoder_tokens

2976

In [80]:
input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])

print(input_token_index)
print(target_token_index)

{'a': 1, 'abandoned': 2, 'ability': 3, 'ablaze': 4, 'able': 5, 'about': 6, 'above': 7, 'abroad': 8, 'absence': 9, 'absent': 10, 'absolute': 11, 'absurd': 12, 'abused': 13, 'accepted': 14, 'access': 15, 'accident': 16, 'accidental': 17, 'accompanied': 18, 'accompany': 19, 'according': 20, 'account': 21, 'accountable': 22, 'accused': 23, 'accustomed': 24, 'ache': 25, 'acknowledgement': 26, 'acquaintance': 27, 'acquaintances': 28, 'acquainted': 29, 'across': 30, 'act': 31, 'actions': 32, 'actor': 33, 'actress': 34, 'add': 35, 'adding': 36, 'address': 37, 'admit': 38, 'adopted': 39, 'advantage': 40, 'advice': 41, 'advise': 42, 'advised': 43, 'affected': 44, 'afford': 45, 'afraid': 46, 'africa': 47, 'after': 48, 'afternoon': 49, 'again': 50, 'against': 51, 'age': 52, 'ago': 53, 'agree': 54, 'agreement': 55, 'aids': 56, 'air': 57, 'airport': 58, 'alarm': 59, 'alcohol': 60, 'alike': 61, 'alive': 62, 'all': 63, 'allergic': 64, 'allow': 65, 'allowances': 66, 'allowed': 67, 'almost': 68, 'alone'

In [0]:
reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())

In [82]:
lines = shuffle(lines)
lines.head(10)

,eng,hin
760,thats a good question,START_ यह एक बहुत अच्छा सवाल है। _END
1150,my aunt brought me flowers,START_ मेरी बुआजी मेरे लिए फूल लाईं। _END
2431,a big earthquake occurred in india yesterday,START_ भारत में कल एक बड़ा भूकम्प हुआ था। _END
2732,the fisherman exaggerated the size of the fish...,START_ मछवारे ने पकड़ी हुई मछली के आकार को बढ़...
15,goodbye,START_ ख़ुदा हाफ़िज़। _END
890,he was then a boy of ten,START_ वह दस साल का लड़का था। _END
1414,he looked into the boys eyes,START_ उसने लड़के की आँखों में देखा। _END
2614,what number should i call in case of an emergency,START_ आपातकालीन स्तिथि में मुझे क्या नम्बर मि...
1494,do you have one thats cheaper,START_ इससे सस्ता वाला है क्या _END
2599,i know her by sight but ive never spoken to her,START_ मैं उसे पहचानता हूँ पर मैंने उससे कभी ब...


In [83]:
# Train - Test Split
X, y = lines.eng, lines.hin
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1)
X_train.shape, X_test.shape

((2500,), (278,))

#### Save the train and test dataframes for reproducing the results later, as they are shuffled.

In [0]:
X_train.to_pickle('/content/drive/My Drive/Eng-Hin-Translator/X_train.pkl')
X_test.to_pickle('/content/drive/My Drive/Eng-Hin-Translator/X_test.pkl')

In [0]:
def generate_batch(X = X_train, y = y_train, batch_size=batch_size):
    ''' Generate a batch of data '''
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_length_src),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_length_tar),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_length_tar, num_decoder_tokens),dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t] = input_token_index[word] # encoder input seq
                for t, word in enumerate(target_text.split()):
                    if t<len(target_text.split())-1:
                        decoder_input_data[i, t] = target_token_index[word] # decoder input seq
                    if t>0:
                        # decoder target sequence (one hot encoded)
                        # does not include the START_ token
                        # Offset by one timestep
                        decoder_target_data[i, t - 1, target_token_index[word]] = 1.
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

### Encoder - Decoder Model Architecture

In [0]:
latent_dim = 50


In [0]:
# Encoder
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(num_encoder_tokens, latent_dim, mask_zero = True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [0]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [0]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])

In [0]:
train_samples = len(X_train)
val_samples = len(X_test)
batch_size = 128
epochs = 200

In [100]:
model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),
                    steps_per_epoch = train_samples//batch_size,
                    epochs=epochs,
                    validation_data = generate_batch(X_test, y_test, batch_size = batch_size),
                    validation_steps = val_samples//batch_size)

Epoch 1/200
19/19 [==============================] - 3s 178ms/step - loss: 1.0146 - acc: 0.3236 - val_loss: 1.4522 - val_acc: 0.2313
Epoch 2/200
19/19 [==============================] - 3s 184ms/step - loss: 0.9847 - acc: 0.3264 - val_loss: 1.5033 - val_acc: 0.2098
Epoch 3/200
19/19 [==============================] - 3s 163ms/step - loss: 0.9787 - acc: 0.3289 - val_loss: 0.3186 - val_acc: 0.2368
Epoch 4/200
19/19 [==============================] - 3s 157ms/step - loss: 0.9724 - acc: 0.3316 - val_loss: 1.4475 - val_acc: 0.2344
Epoch 5/200
19/19 [==============================] - 3s 164ms/step - loss: 0.9659 - acc: 0.3337 - val_loss: 1.5037 - val_acc: 0.2098
Epoch 6/200
19/19 [==============================] - 3s 165ms/step - loss: 0.9628 - acc: 0.3368 - val_loss: 0.3164 - val_acc: 0.2359
Epoch 7/200
19/19 [==============================] - 3s 150ms/step - loss: 0.9541 - acc: 0.3400 - val_loss: 1.4505 - val_acc: 0.2318
Epoch 8/200
19/19 [==============================] - 3s 162ms/step - 

### Always remember to save the weights

In [0]:
model.save_weights('/content/drive/My Drive/Eng-Hin-Translator/nmt_weights.h5')

### Load the weights, if you close the application

In [0]:
model.load_weights('/content/drive/My Drive/Eng-Hin-Translator/nmt_weights.h5')

### Inference Setup

In [0]:
# Encode the input sequence to get the "thought vectors"
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2= dec_emb_layer(decoder_inputs) # Get the embeddings of the decoder sequence

# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2) # A dense softmax layer to generate prob dist. over the target vocabulary

# Final decoder model
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

### Decode sample sequeces

In [0]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['START_']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or
           len(decoded_sentence) > 50):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

### Evaluation on Train Dataset

In [0]:
train_gen = generate_batch(X_train, y_train, batch_size = 1)
k=-1

In [120]:
for i in range(20): 
  k+=1
  (input_seq, actual_output), _ = next(train_gen)
  decoded_sentence = decode_sequence(input_seq)
  print('Input English sentence:', X_train[k:k+1].values[0])
  print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
  print('Predicted Hindi Translation:', decoded_sentence[:-4])
  print('\n')

Input English sentence: your dog is very big
Actual Hindi Translation:  तुम्हारा कुत्ता बहुत बड़ा है। 
Predicted Hindi Translation:  तुम्हारा कुत्ता बहुत बड़ा है। 


Input English sentence: i spent the whole afternoon chatting with friends
Actual Hindi Translation:  मैंने दोपहरभर अपने दोस्तों से गप मारी। 
Predicted Hindi Translation:  मैंने अपने दोस्तों से खबर से गया। 


Input English sentence: the plane will take off in one hour
Actual Hindi Translation:  विमान एक घंटे में उड़ान भरेगा। 
Predicted Hindi Translation:  विमान एक घंटे में उड़ान भरेगा। 


Input English sentence: something is wrong with this washing machine
Actual Hindi Translation:  इस वॉशिंग मशीन में कुछ खराबी है। 
Predicted Hindi Translation:  इस वॉशिंग मशीन में कुछ खराबी है। 


Input English sentence: my sister has a job
Actual Hindi Translation:  मेरी बहन के पास नौकरी है। 
Predicted Hindi Translation:  मेरी बहन के पास नौकरी है। 


Input English sentence: have fun
Actual Hindi Translation:  मज़े करना। 
Predicted Hindi Tr

### Evaluation on Validation Dataset

In [0]:
val_gen = generate_batch(X_test, y_test, batch_size = 1)
k=-1

In [124]:
for i in range(20): 
  k+=1
  (input_seq, actual_output), _ = next(val_gen)
  decoded_sentence = decode_sequence(input_seq)
  print('Input English sentence:', X_test[k:k+1].values[0])
  print('Actual Marathi Translation:', y_test[k:k+1].values[0][6:-4])
  print('Predicted Marathi Translation:', decoded_sentence[:-4])
  print('\n')

Input English sentence: let me give you a piece of advice
Actual Marathi Translation:  मैं तुम्हें एक बात की सलाह देता हूँ। 
Predicted Marathi Translation:  मुझे तुम्हें काम के साथ गरम चहिए। 


Input English sentence: a police officer is talking to her
Actual Marathi Translation:  एक पुलिस अफ़सर उससे बात कर रहा है। 
Predicted Marathi Translation:  वह सब समझ में ज़्यादा समय नहीं जा रहे 


Input English sentence: today is my sixteenth birthday
Actual Marathi Translation:  आज मेरा सोलहवा जन्मदिन है। 
Predicted Marathi Translation:  आज आज किताब पढ़ रहे हैं। 


Input English sentence: i know a girl who can ride a unicycle
Actual Marathi Translation:  मेरे जानपहचान की एक लड़की को युनिसायकल चलाना आता है। 
Predicted Marathi Translation:  मुझे यह पता है कि यह तो क्या पहनूँ। 


Input English sentence: i met her in london for the first time
Actual Marathi Translation:  मेरी उसके साथ पहली मुलाकात लंदन में हुई थी। 
Predicted Marathi Translation:  मैं अपने घर में पहले खतम किया। 


Input English sent